# Integration test to check that all latest modules run

In [ ]:
import json
import os
import sys
import tarfile
from glob import glob
import re

from collections import Counter
import requests
from datetime import datetime
from pathlib import Path

import tengu

### 0) Setup

In [ ]:
# Set our token - ensure you have exported TENGU_TOKEN in your shell; or just replace the os.getenv with your token
TOKEN = os.getenv("TENGU_TOKEN")
URL = os.getenv("TENGU_URL")

In [ ]:
# Get our client, for calling modules and using the tengu API
client = tengu.Provider(access_token=TOKEN, url=URL)

In [ ]:
modules = await client.get_latest_module_paths()

In [ ]:
modules

{'gnina_pdb': 'github:talo/tengu-gnina/6f97cbd5df44535f439a9f94bb3af3d5d72c933c#gnina_tengu_pdb',
 'gnina': 'github:talo/tengu-gnina/6f97cbd5df44535f439a9f94bb3af3d5d72c933c#gnina_tengu',
 'fragment_aa': 'github:talo/tengu-prelude/e21ba62b15154b9632d29238e9677749febfd088#fragment_aa',
 'fragment': 'github:talo/tengu-prelude/e21ba62b15154b9632d29238e9677749febfd088#fragment',
 'cairo': 'github:talo/cairo/a946052d1004e6cbb343f8001c46b122c3a317f1#cairo_tengu',
 'delay': 'github:talo/tengu-module-example/dd0001bab86d908b09aae8187421da23a9b35d78#delay',
 'hermes_binding': 'github:talo/tengu-prelude/083e958d64e9e716754f76560bd3e60557e6a691#hermes_binding',
 'gmx_frame_select_pdb': 'github:talo/gmx_tengu_support/68cfc9dc1533bd54dfa21b346642410ff0fce570#gmx_frame_select_pdb',
 'gmx_frame_select': 'github:talo/gmx_tengu_support/a15ebf3e86b779203b463193c0ecc025c9f916df#gmx_frame_select',
 'gmx_mmpbsa': 'github:talo/gmx_tengu_support/d4ea797dcdedf9b91de1b76a32f8a95f0cbf21df#gmx_mmpbsa_tengu',
 'e

In [ ]:
deploy_instances = [next(client.module_instances(tags=["deploy"], path=path, status="COMPLETED")) for path in modules.values()]

In [ ]:
deploy_instances = [anext(await client.module_instances(tags=["deploy"], path=path, status="COMPLETED")) for path in modules.values()]
for instance_set in deploy_instances:
    x = await instance_set
    for instance in x.edges: 
        instance = instance.node
        if not "retry" in str.join("", instance.tags):
            retry = await client.retry(instance.id, target=instance.target)
            print("retrying", instance.id, "as", retry)

retrying 49b55de3-91e2-4e99-a156-e0039c2971c1 as id=UUID('37b85724-016f-4b0d-afb1-f28cb34d5fa0')
retrying 30d0bb4e-0cb1-473c-ad34-e6fe87ff5ca4 as id=UUID('7506fe8d-f81c-4fb6-b17e-85f2a943ea64')
retrying 40544481-39e1-447b-b1a0-6cc8f479f2b4 as id=UUID('0a297df3-ae83-4da8-aa07-ac807957c041')
retrying fe8ce2a1-8e80-451a-842a-ed9afdb8d116 as id=UUID('ea927d2b-1edb-4541-b92c-5919a3dbc372')
retrying 450eee25-f03d-4e73-88b6-4f8143a7b386 as id=UUID('6347e684-23e3-493a-8596-c40a71285e26')
retrying d6e2164e-bac3-4176-b755-a88233140b98 as id=UUID('cdcfa127-0073-46c7-9e72-5b0ab5d36eb4')
retrying eaa1520c-b364-4114-b74b-edaf1547e103 as id=UUID('e0db451a-18ed-4eee-b3ab-7c57ff68c6d5')
retrying 3ac69002-d2b8-41ea-a3bc-1d1eef2d3d8f as id=UUID('d6a8d8ae-89df-41cc-a100-0dd69116d05d')
retrying ad61f484-6f68-4d9a-8680-2f281309608c as id=UUID('97a70db8-8c4e-4bcd-820f-feef7e17cc97')
retrying cc0800c8-d3a0-4165-a826-1a2f93e9e8ac as id=UUID('89b88af0-1ed8-4a6e-b0b0-582a6dc39e82')
retrying fc4c034b-f5eb-453f-9d

In [ ]:
retry_instances = (await client.module_instances(tags=["deploy", "retry_0"], path=path) for path in modules.values())
instance_pages = [instance.edges async for page in retry_instances async for instance in page]
c = Counter([(instance.node.status, instance.node.path.split("#")[1], instance.node.target) for page in instance_pages for instance in page])
print(f"{'Module':<10} | {'Status':<20} | {'Target':<20} | Count")
print("-" * 50)
for (module, status, target), count in c.items():
    print(f"{module:<10} | {status:<20} | {target:<20} | {count}")

Module     | Status               | Target               | Count
--------------------------------------------------
QUEUED     | gnina_tengu_pdb      | NIX_SSH_3_GPU        | 1
COMPLETED  | gnina_tengu_pdb      | NIX_SSH_3_GPU        | 2
ADMITTED   | gnina_tengu          | NIX_SSH_3_GPU        | 1
COMPLETED  | gnina_tengu          | NIX_SSH_3_GPU        | 1
AWAITING_UPLOAD | fragment_aa          | NIX                  | 1
COMPLETED  | fragment_aa          | NIX                  | 1
AWAITING_UPLOAD | fragment             | NIX                  | 1
COMPLETED  | fragment             | NIX                  | 1
ADMITTED   | cairo_tengu          | GADI_GPU             | 2
COMPLETED  | cairo_tengu          | GADI_GPU             | 2
COMPLETED  | delay                | NIX                  | 2
DISPATCHED | hermes_binding       | NIX_SSH_GPU          | 1
COMPLETED  | hermes_binding       | NIX_SSH_GPU          | 3
COMPLETED  | hermes_binding       | NIX_SSH              | 3
COMPLETED  | gmx_fra

In [ ]:
async for i in await client.module_instances():
    print(i)

edges=[ModuleInstancesMeAccountModuleInstancesEdges(cursor='F6IxgI2dqqA=', node=ModuleInstancesMeAccountModuleInstancesEdgesNode(id=UUID('8415b1c5-7212-46ee-a8c7-5261878451ca'), account_id=UUID('e89afcd8-34f1-4f40-b292-84fbf875c3cd'), created_at=datetime.datetime(2023, 12, 19, 9, 27, 17, 300964, tzinfo=TzInfo(UTC)), deleted_at=None, queued_at=datetime.datetime(2023, 12, 19, 9, 27, 17, 300968, tzinfo=TzInfo(UTC)), admitted_at=datetime.datetime(2023, 12, 19, 9, 27, 25, 925521, tzinfo=TzInfo(UTC)), dispatched_at=datetime.datetime(2023, 12, 19, 9, 27, 30, 904188, tzinfo=TzInfo(UTC)), completed_at=datetime.datetime(2023, 12, 19, 9, 27, 37, 210331, tzinfo=TzInfo(UTC)), path='github:talo/tengu-prelude/9b0c89cbd3b7541b6d700ccf66bbdf9e4c82b630#concat', status=<ModuleInstanceStatus.COMPLETED: 'COMPLETED'>, target=<ModuleInstanceTarget.NIX: 'NIX'>, tags=['deploy', 'retry_0'], ins=[SimpleModuleInstanceFullIns(id=UUID('d32ca175-98b1-485d-b368-fedb6fc63e87'), created_at=datetime.datetime(2023, 10, 3